In [ ]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms

from context import rf_pool

**Load MNIST Data**

In [ ]:
# get MNIST training data
transform = transforms.Compose([transforms.ToTensor()])
trainset = torchvision.datasets.MNIST(root='../data', train=True, download=True, transform=transform)

In [ ]:
# create trainloader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1,
                                          shuffle=True, num_workers=2)

**Initialize model with RF layer**

In [ ]:
# initialize model
model = rf_pool.models.FeedForwardNetwork()

In [ ]:
# create 3x3 uniform lattice
img_shape = torch.tensor([24,24])
mu, sigma = rf_pool.utils.lattice.init_uniform_lattice(img_shape // 2, (3,3), 6, 1.)
rf_layer = rf_pool.pool.MaxPool((2,2), mu, sigma, img_shape)
rf_layer.show_lattice()
print(mu.shape)
n_kernels = mu.shape[0]

**Create control network to update RF locations and sizes**

In [ ]:
# set control network for updating mu/sigma
control_net = rf_pool.models.FeedForwardNetwork()
control_net.append('0', rf_pool.modules.FeedForward(hidden=torch.nn.Conv2d(16,32,10), activation=torch.nn.ReLU()))
control_net.append('1', rf_pool.modules.FeedForward(input_shape=(-1,32*15*15), hidden=torch.nn.Linear(32*15*15,128), 
                                                 activation=torch.nn.ReLU()))
branch0 = rf_pool.modules.FeedForward(hidden=torch.nn.Linear(128,n_kernels*2), activation=torch.nn.Tanh())
branch1 = rf_pool.modules.FeedForward(hidden=torch.nn.Linear(128,n_kernels), activation=torch.nn.Tanh())
control_net.append('2', rf_pool.modules.Branch([branch0, branch1],
                                                branch_shapes=[(n_kernels,2), (n_kernels,1)]))

In [ ]:
# append layers of model
model.append('0', rf_pool.modules.Control(hidden=torch.nn.Conv2d(1,16,5), activation=torch.nn.ReLU(), 
                                          control=control_net, pool=rf_layer))
model.append('1', rf_pool.modules.FeedForward(hidden=torch.nn.Conv2d(16,32,5), activation=torch.nn.ReLU(),
                                              pool=torch.nn.MaxPool2d(2)))
model.append('2', rf_pool.modules.FeedForward(hidden=torch.nn.Conv2d(32,64,4)))
model.append('3', rf_pool.modules.FeedForward(input_shape=(-1, 64), hidden=torch.nn.Linear(64, 10)))

In [ ]:
# get base parameters separate from control parameters
model.set_requires_grad(pattern='control_layers', requires_grad=False)
base_params = model.get_trainable_params()
model.set_requires_grad(pattern='control_layers', requires_grad=True)

**Train model and control network to classify digits**

In [ ]:
# set loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)

In [ ]:
# train model, monitor weights and lattice
loss_history = model.train(1, trainloader, loss_fn, optim, monitor=100,
                           show_weights={'layer_id': '0', 'cmap': 'gray'},
                           show_lattice={'cmap': 'gray'})